In [3]:
from qiskit import QuantumCircuit, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity

In [5]:
# =====================================================
# 1. Original entangled state on qubits (0,1)
# =====================================================

orig_circ = QuantumCircuit(3)
orig_circ.h(0)
orig_circ.cx(0, 1)

# Save the full 3-qubit density matrix
orig_circ.save_density_matrix(label="rho_orig")

sim_dm = AerSimulator(method="density_matrix")
orig_compiled = transpile(orig_circ, sim_dm)
orig_result = sim_dm.run(orig_compiled).result()

rho_full_orig = DensityMatrix(orig_result.data(0)["rho_orig"])
# Reduced density matrix for qubits (0,1)
rho_01 = partial_trace(rho_full_orig, [2])  # trace out qubit 2


# =====================================================
# 2. Teleportation circuit with measurement + feed-forward
# =====================================================

# One classical bit for s (X-basis measurement of q1)
tele_circ_dm = QuantumCircuit(3, 1)

# Prepare entangled state on (0,1)
tele_circ_dm.h(0)
tele_circ_dm.cx(0, 1)

# Prepare q2 in |+>
tele_circ_dm.h(2)

# Apply CZ between q1 and q2
tele_circ_dm.cz(1, 2)

# Measure q1 in X basis: H then Z-basis measure
tele_circ_dm.h(1)
tele_circ_dm.measure(1, 0)  # store s in classical bit 0

# Feed-forward: apply X^s H on qubit 2
with tele_circ_dm.if_test((tele_circ_dm.cregs[0], 1)):
    tele_circ_dm.x(2)
tele_circ_dm.h(2)                                 # then H always

# Save density matrix after teleportation + correction
tele_circ_dm.save_density_matrix(label="rho_final")

tele_compiled_dm = transpile(tele_circ_dm, sim_dm)
tele_result_dm = sim_dm.run(tele_compiled_dm).result()

rho_full_final = DensityMatrix(tele_result_dm.data(0)["rho_final"])
# Reduced density matrix for qubits (0,2)
rho_02 = partial_trace(rho_full_final, [1])  # trace out qubit 1

In [6]:
# =====================================================
# 3. Fidelity between original (0,1) and teleported (0,2)
# =====================================================

F = state_fidelity(rho_01, rho_02)
print("Fidelity between original (q0,q1) and teleported (q0,q2):", F)

Fidelity between original (q0,q1) and teleported (q0,q2): 1.0000000000000004
